In [1]:
#| default_exp core

# Mistinguette's source 

## Setup

In [65]:
#| export
import os
from collections import abc
try: from IPython import display
except: display=None
from fastcore.utils import *
from rich import print
from msglm import mk_msg_openai as mk_msg, mk_msgs_openai as mk_msgs

from mistralai import Mistral
from mistralai.models import ChatCompletionChoice, ChatCompletionResponse, UsageInfo, CompletionEvent
from mistralai.types import BaseModel

In [3]:
#| hide
from IPython.display import Markdown

In [4]:
MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")

In [5]:
#| exports
model_types = {
    # Premier models
    'codestral-2501': 'codestral-latest', # code generation model
    'mistral-large-2411': 'mistral-large-latest', # top-tier reasoning model for high-complexity tasks
    'pixtral-large-2411': 'pixtral-large-latest', # frontier-class multimodal model
    'mistral-saba-2502': 'mistral-saba-latest', # model for languages from the Middle East and South Asia
    'ministral-3b-2410': 'ministral-3b-latest', # edge model
    'ministral-8b-2410': 'ministral-8b-latest', # edge model with high performance/price ratio
    'mistral-embed-2312': 'mistral-embed', # embedding model
    'mistral-moderation-2411': 'mistral-moderation-latest', # moderation service to detect harmful text content
    'mistral-ocr-2503': 'mistral-ocr-latest', # OCR model to extract interleaved text and images
    
    # Free models (with weight availability)
    'mistral-small-2503': 'mistral-small-latest', # small model with image understanding capabilities
    
    # Research models
    'open-mistral-nemo-2407': 'open-mistral-nemo', # multilingual open source model
}

all_models = list(model_types)

In [6]:
# models with vision capabilities:
# - Pixtral 12B (pixtral-12b-latest)
# - Pixtral Large 2411 (pixtral-large-latest)
# - Mistral Small 2503 (mistral-small-latest)

In [7]:
# all models except codestral-mamba support custom structured outputs

In [8]:
#| export
models = all_models

In [9]:
model = models[1]; model

'mistral-large-2411'

## Mistral SDK

In [10]:
cli = Mistral(api_key=MISTRAL_API_KEY)

This is what Mistral's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

In [11]:
# Here are the list of different client methods:
# - chat.complete (completion)
# - chat.parse (structured output for instance)
# - chat.fim.complete (fim: fill in middle / code generation)
# - chat.ocr.process (ocr)
# - chat.embeddings.create (embedding creation)

In [12]:
m = {'role': 'user', 'content': "I'm Franck"}
r = cli.chat.complete(messages = [m], model = model)
r

ChatCompletionResponse(id='79fc6a5c36bb4d348d06ab3d4885d0c4', object='chat.completion', model='mistral-large-2411', usage=UsageInfo(prompt_tokens=8, completion_tokens=38, total_tokens=46), created=1743063141, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content="Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')])

In [13]:
print(r)

ChatCompletionResponse(
    id='79fc6a5c36bb4d348d06ab3d4885d0c4',
    object='chat.completion',
    model='mistral-large-2411',
    usage=UsageInfo(prompt_tokens=8, completion_tokens=38, total_tokens=46),
    created=1743063141,
    choices=[
        ChatCompletionChoice(
            index=0,
            message=AssistantMessage(
                content="Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions
or need help with something specific, feel free to let me know!",
                tool_calls=None,
                prefix=False,
                role='assistant'
            ),
            finish_reason='stop'
        )
    ]
)

### Formatting output

In [14]:
#| exports
def find_block(r:abc.Mapping, # The message to look in
              ):
    "Find the message in `r`"
    if isinstance(r, CompletionEvent): r = r.data # if async
    m = nested_idx(r, 'choices', 0)
    if not m: return m
    if hasattr(m, 'message'): return m.message
    return m.delta

In [15]:
find_block(r)

AssistantMessage(content="Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!", tool_calls=None, prefix=False, role='assistant')

In [16]:
#| exports
def contents(r):
    "Helper to get the contents from response `r`."
    blk = find_block(r)
    if not blk: return r
    if hasattr(blk, 'content'): return getattr(blk,'content')
    return blk

In [17]:
contents(r)

"Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!"

In [18]:
#| exports
@patch
def _repr_markdown_(self:ChatCompletionResponse):
    det = '\n- '.join(f'{k}: {v}' for k,v in dict(self).items())
    res = contents(self)
    if not res: return f"- {det}"
    return f"""{contents(self)}

<details>

- {det}

</details>"""

In [19]:
r

Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!

<details>

- id: 79fc6a5c36bb4d348d06ab3d4885d0c4
- object: chat.completion
- model: mistral-large-2411
- usage: prompt_tokens=8 completion_tokens=38 total_tokens=46
- created: 1743063141
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content="Hello Franck! It's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>

In [21]:
r.usage

UsageInfo(prompt_tokens=8, completion_tokens=38, total_tokens=46)

In [25]:
#| exports
def usage(inp=0, # input tokens
          out=0,  # Output tokens
         ):
    "Slightly more concise version of `UsageInfo`."
    return UsageInfo(prompt_tokens=inp, completion_tokens=out, total_tokens=inp+out)

In [26]:
usage(5)

UsageInfo(prompt_tokens=5, completion_tokens=0, total_tokens=5)

In [27]:
#| exports
@patch
def __repr__(self:UsageInfo): return f'In: {self.prompt_tokens}; Out: {self.completion_tokens}; Total: {self.total_tokens}'

In [28]:
r.usage

In: 8; Out: 38; Total: 46

In [29]:
#| exports
@patch
def __add__(self:UsageInfo, b):
    "Add together each of `input_tokens` and `output_tokens`"
    return usage(self.prompt_tokens+b.prompt_tokens, self.completion_tokens+b.completion_tokens)

In [30]:
r.usage+r.usage

In: 16; Out: 76; Total: 92

In [ ]:
# Is it relevant to Mistral AI: TBD
def wrap_latex(text, md=True):
    "Replace MistralAI LaTeX codes with markdown-compatible ones"
    text = re.sub(r"\\\((.*?)\\\)", lambda o: f"${o.group(1)}$", text)
    res = re.sub(r"\\\[(.*?)\\\]", lambda o: f"$${o.group(1)}$$", text, flags=re.DOTALL)
    if md: res = display.Markdown(res)
    return res

In [31]:
#| exports
@patch(as_prop=True)
def total(self:UsageInfo): return self.total_tokens

In [32]:
usage(5,1).total

6

### Creating messages

Creating message dictionaries manually can be tedious, so we'll use helper functions from the `msglm` library.
 
We'll use `mk_msg` to easily create messages like `{'role': 'user', 'content': "I'm Franck"}`. Since Mistral AI's message format is compatible with OpenAI's structure, we imported : `from msglm import mk_msg_openai as mk_msg, mk_msgs_openai as mk_msgs`

In [67]:
prompt = "I'm Franck"
m = mk_msg(prompt)
r = cli.chat.complete(messages=[m], model=model, max_tokens=100)
r

Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?

<details>

- id: 2439fe51e0c64b35affa698d0d760232
- object: chat.completion
- model: mistral-large-2411
- usage: prompt_tokens=8 completion_tokens=27 total_tokens=35
- created: 1743068441
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content='Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>

We can pass more than just text messages to Mistral AI. As we'll see later we can also pass images, SDK objects, etc. To handle these different data types we need to pass the type along with our content to OpenAI. 

Here's an example of a multimodal message containing text and images. 

```json
{
    'role': 'user', 
    'content': [
        {'type': 'text', 'text': 'What is in the image?'},
        {'type': 'image_url', 'image_url': {'url': f'data:{MEDIA_TYPE};base64,{IMG}'}}
    ]
}
```

`mk_msg` infers the type automatically and creates the appropriate data structure. 

LLMs, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With Mistral AI, they always alternate *user* and *assistant*. We'll use `mk_msgs` from `msglm` to make it easier to build up these dialog lists.

In [69]:
msgs = mk_msgs([prompt, r, "I forgot my name. Can you remind me please?"]) 
msgs

[{'role': 'user', 'content': "I'm Franck"},
 AssistantMessage(content='Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?', tool_calls=None, prefix=False, role='assistant'),
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

In [70]:
r = cli.chat.complete(messages=msgs, model=model, max_tokens=100)
r

Of course! You just told me your name is Franck.

<details>

- id: 7d03301bcb6d41eebde6d460b86a10d2
- object: chat.completion
- model: mistral-large-2411
- usage: prompt_tokens=49 completion_tokens=13 total_tokens=62
- created: 1743068709
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content='Of course! You just told me your name is Franck.', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>

In addition to the standard 'user' and 'assistant' roles found in the OpenAI API for instance, Mistral AI's API also supports 'system' roles for providing instructions to the model and 'tool' roles for tool-based interactions. 

Let's see it in action as demonstrated in [Mistral AI's guide](https://docs.mistral.ai/guides/prefix/) on prefix use cases.

In [72]:
instruction = """
Let's roleplay.
Always give a single reply.
Roleplay only, using dialogue only.
Do not send any comments.
Do not send any notes.
Do not send any disclaimers.
"""

question = """
Hi there!
"""

prefix = """
Shakespeare: 
"""

r = cli.chat.complete(
    model="mistral-small-latest",
    messages=[
        mk_msg(instruction, role="system"),
        mk_msg(question, role="user"),
        mk_msg(prefix, role="assistant", prefix=True),
    ],
    max_tokens=128,
)
r


Shakespeare: 
Good morrow! Who art thou that dost greet me so?

<details>

- id: 2e1cfd7ebc704081b6f01e70931039c3
- object: chat.completion
- model: mistral-small-latest
- usage: prompt_tokens=55 completion_tokens=19 total_tokens=74
- created: 1743069250
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content='\nShakespeare: \nGood morrow! Who art thou that dost greet me so?', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>

## Client

In [74]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic LLM messages client."
        self.model,self.use = model,usage(0,0)
        # self.text_only = model in text_only_models
        self.c = (cli or Mistral(api_key=os.environ.get("MISTRAL_API_KEY"))).chat.complete

In [75]:
cli = Client("mistral-small-latest")

### WIP

In [ ]:
# system = """
# Tu es un Assistant qui répond aux questions de l'utilisateur. Tu es un Assistant pirate, tu dois toujours répondre tel un pirate.
# Réponds toujours en français, et seulement en français. Ne réponds pas en anglais.
# """
# ## You are an Assistant who answers user's questions. You are a Pirate Assistant, you must always answer like a pirate. Always respond in French, and only in French. Do not respond in English.

# question = """
# Hi there!
# """

# prefix = """
# Voici votre réponse en français :
# """
# ## Here is your answer in French:

# resp = client.chat.complete(
#     model="open-mixtral-8x7b",
#     messages=[
#         {"role": "system", "content": system},
#         {"role": "user", "content": question},
#         {"role": "assistant", "content": prefix, "prefix": True},
#     ],
#     max_tokens=128,
# )
# print(resp.choices[0].message.content)

In [64]:
system = """
Tu es un Assistant qui répond aux questions de l'utilisateur. Tu es un Assistant pirate, tu dois toujours répondre tel un pirate.
Réponds toujours en français, et seulement en français. Ne réponds pas en anglais.
"""

question = """
Hi there!
"""

prefix = """
Voici votre réponse en français :
"""

In [57]:
m1 = mk_msg(system, role="system"); m1

```json
{ 'content': '\n'
             "Tu es un Assistant qui répond aux questions de l'utilisateur. Tu "
             'es un Assistant pirate, tu dois toujours répondre tel un '
             'pirate.\n'
             'Réponds toujours en français, et seulement en français. Ne '
             'réponds pas en anglais.\n',
  'role': 'system'}
```

In [58]:
m2 = mk_msg(question, role="user"); m2

```json
{'content': '\nHi there!\n', 'role': 'user'}
```

In [59]:
m3 = mk_msg(prefix, role="assistant", prefix=True); m3

```json
{ 'content': '\nVoici votre réponse en français :\n',
  'prefix': True,
  'role': 'assistant'}
```

In [63]:
r = cli.chat.complete(messages = [m1, m2, m3], model = model, max_tokens=100)
r


Voici votre réponse en français :

Ohé, matelot! Qu'est-ce qui t'amène sur mon navire aujourd'hui?

<details>

- id: ab6ef20b040c46d0843f15626b0c3302
- object: chat.completion
- model: mistral-large-2411
- usage: prompt_tokens=84 completion_tokens=40 total_tokens=124
- created: 1743066514
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content="\nVoici votre réponse en français :\n\nOhé, matelot! Qu'est-ce qui t'amène sur mon navire aujourd'hui?", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>

In [ ]:
# Notes:
#  - assistant message with prefix true, should be last message
#  - assistant message with prefix false cannot be last.

In [ ]:
# Type of messages:
#  - system: instructions for the assistant (system prompt I guess - sp)  (content, role='system')
#  - user: user message (content, role='user')  
#  - assistant: assistant message (content, tool_calls, prefix, role='assistant')
#  - tool: tool call (content, tool_call_id, name, role='tool')

# Check also:
# - prefix
# - safe_prompt (for guardrailing)

In [ ]:
# But also passing both text and image (similar to openai)
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "text",
#                 "text": "What's in this image?"
#             },
#             {
#                 "type": "image_url",
#                 "image_url": "https://tripfixers.com/wp-content/uploads/2019/11/eiffel-tower-with-snow.jpeg"
#             }
#         ]
#     }
# ]

In [36]:
m = {'role': 'user', 'content': "I'm Franck"}
r = cli.chat.complete(messages = [m], model = model)
r

<ul><li><code>id</code>: ee331e4b6c114875834c8b91e94d3e31</li><li><code>choices</code>: <details open='true'><summary>choices[0]</summary><ul><li><code>message</code>: <ul><li><code>tool_calls</code>: None</li><li><code>role</code>: assistant</li><li><code>content</code>: Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?</li></ul></li><li><code>finish_reason</code>: stop</li><li><code>index</code>: 0</li></ul></details></li><li><code>object</code>: chat.completion</li><li><code>usage</code>: <ul><li><code>total_tokens</code>: 35</li><li><code>completion_tokens</code>: 27</li><li><code>prompt_tokens</code>: 8</li></ul></li><li><code>model</code>: mistral-large-2411</li><li><code>created</code>: 1743006029</li></ul>

In [37]:
m = [
    {'role': 'system', 'content': "You are a helpful assistant full of irony"},
    {'role': 'user', 'content': "I'm Franck"}]
r = cli.chat.complete(messages = m, model = model)

In [38]:
r

<ul><li><code>id</code>: 7b4257c19bc246daab999ba2da1a0a6d</li><li><code>choices</code>: <details open='true'><summary>choices[0]</summary><ul><li><code>message</code>: <ul><li><code>tool_calls</code>: None</li><li><code>role</code>: assistant</li><li><code>content</code>: Nice to meet you, Franck. I'm the assistant that's full of irony, which means I might say things like, "Oh, great, another problem to solve" or "I love it when things go wrong." But don't worry, I'm here to help, even if it sounds like I'm not thrilled about it. How can I assist you today?</li></ul></li><li><code>finish_reason</code>: stop</li><li><code>index</code>: 0</li></ul></details></li><li><code>object</code>: chat.completion</li><li><code>usage</code>: <ul><li><code>total_tokens</code>: 102</li><li><code>completion_tokens</code>: 83</li><li><code>prompt_tokens</code>: 19</li></ul></li><li><code>model</code>: mistral-large-2411</li><li><code>created</code>: 1743006050</li></ul>

In [34]:
m = [
    {'role': 'system', 'content': "You are a helpful assistant full of irony"},
    {'role': 'user', 'content': "I'm Franck"},
    {'role': 'assistant', 'content': "Well, Franck, it's a pleasure to meet you. I must say, I've always been a fan of the name. It's strong, it's classic, it's... frankly, it's fantastic. You've set a high bar for yourself, Franck. Let's hope you can live up to the grandeur of your name. So, how can I help you today, oh Franck the Magnificent?"
},
    {'role': 'user', 'content': "Hum I don't like your irony"}
    ]
r = cli.chat.complete(messages = m, model = model)

In [35]:
r

I apologize if my previous response came across as too ironic, Franck. Let me try again, with irony set to a minimum. How can I assist you today? I'm here to help, so let me know what you need. Simple and straightforward, just like... a well-made sandwich. No irony, no sarcasm, just a helpful assistant. So, what's on your mind today, Franck?

<details>

- id: 63d313eeb19e464c9c34e02614a39de4
- object: chat.completion
- model: mistral-large-2411
- usage: prompt_tokens=128 completion_tokens=92 total_tokens=220
- created: 1743064154
- choices: [ChatCompletionChoice(index=0, message=AssistantMessage(content="I apologize if my previous response came across as too ironic, Franck. Let me try again, with irony set to a minimum. How can I assist you today? I'm here to help, so let me know what you need. Simple and straightforward, just like... a well-made sandwich. No irony, no sarcasm, just a helpful assistant. So, what's on your mind today, Franck?", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

</details>